## Bring Your Own Container Batch Inference

In this example we take a look at how we can bring an open source XGBoost pre-trained model to Batch Inference.

### Local Model Creation + Training

We will first locally generate a sample xgboost.json artifact to use in our container.

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X, y = datasets.load_diabetes(return_X_y=True) #load sklearn diabetes dataset for training
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


xbg_reg = xgb.XGBRegressor().fit(X_train_scaled, y_train)

In [ ]:
# Save as JSON file
xbg_reg.save_model("model.json")

In [ ]:
# Blank new instance to be loaded into
import xgboost as xgb
xgb_test = xgb.Booster()
xgb_test.load_model("model.json")


#we can implement our health check locally that we will in our container later to verify
health = xgb_test is not None
status = 200 if health else 404
status

#### Create Test Dataset + Local inference

In [ ]:
import pandas as pd 
pd.DataFrame(X_test_scaled).to_csv("diabetes-test.csv", index = False)

In [ ]:
import pandas as pd
test = pd.read_csv("diabetes-test.csv", header = None)
test

In [ ]:
preds = xgb_test.predict(xgb.DMatrix(test))
print(preds[:10])

#### Test the predictor.py code for returning data format locally

In [ ]:
from io import StringIO

out = StringIO()
pd.DataFrame({"results": preds}).to_csv(out, header=False, index=False)

result = out.getvalue().rstrip(
        "\n"
    )

result

### Build ECR Image for XGBoost

Container Structure

- Dockerfile
- XGB
    - model.json (copy this local artifact to the container)
    - nginx.conf (don't adjust)
    - serve (don't adjust)
    - wsgi.py (don't adjust)
    - predictor.py (inference logic here, change as needed)

In [ ]:
%%sh

# Name of algo -> ECR
algorithm_name=sm-pretrained-xgboost

cd container

#make serve executable
chmod +x XGB/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
role = get_execution_role()

sess = sagemaker.Session()
region = boto3.session.Session().region_name
#adjust the string with repository created in previous shell, this is the algo name you defined
image = '{}.dkr.ecr.{}.amazonaws.com/sm-pretrained-xgboost:latest'.format(account_id, region)

### Upload Test Dataset for Batch Inference

In [ ]:
from sagemaker.s3 import S3Uploader
s3_test_path = f"s3://{sess.default_bucket()}/"+"xgb-data-batch"
s3_test_uri = S3Uploader.upload(local_path="diabetes-test.csv",desired_s3_uri=s3_test_path)
print(f"model artifcats uploaded to {s3_test_uri}")

### Create SM Model

In [ ]:
from time import gmtime, strftime

model_name = 'xgb-model' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Model name: ' + model_name)

container = {
    'Image': image
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

### Batch Transform Job

In [ ]:
s3_output_path = f"s3://{sess.default_bucket()}/"+"xgb-output-batch"
s3_output_path

In [ ]:
data_output={
        'S3OutputPath': s3_output_path,
    }


data_input={
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': s3_test_uri
            }
        },
        'ContentType': 'text/csv',
        'SplitType': 'Line'
    }

hardware_resources={
        'InstanceType': 'ml.m4.4xlarge',
        'InstanceCount': 1
    }

In [ ]:
%%time
transform_name = 'xgb-model-transform' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
response = sm_client.create_transform_job(TransformJobName = transform_name, ModelName = model_name, 
                                          TransformInput = data_input, TransformOutput = data_output,
                                          TransformResources = hardware_resources)

In [ ]:
# wait for transform job to reach a terminal state (completed)
import time

describe_transform_response = sm_client.describe_transform_job(TransformJobName = transform_name)

while describe_transform_response["TransformJobStatus"] == "InProgress":
    describe_transform_response = sm_client.describe_transform_job(TransformJobName = transform_name)
    print(describe_transform_response["TransformJobStatus"])
    time.sleep(30)

describe_transform_response

In [ ]:
results = describe_transform_response['TransformOutput']['S3OutputPath']
results #capture your results in this S3 location